In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo, fci

I built this script to check on one of the terms in the script me and steffen have been trying to write. 

$ \langle ij|ab\rangle\langle jk|bc\rangle\langle ki|ca\rangle $

$= [ia|jb][jb|kc][kc|ia]$

In [2]:
def enumerator(eri,i,j,k,a,b,c):
    return eri[i,a,j,b]*eri[j,b,k,c]*eri[k,c,i,a]

$( \epsilon_{j} - \epsilon_{b} - \epsilon_{i} + \epsilon_{a})(\epsilon_{k} - \epsilon_{c} - \epsilon_{i} +\epsilon_{a})$

In [3]:
def denominator_1(eigval,i,j,k,a,b,c):
    return ((eigval[j]-eigval[b]-eigval[i]+eigval[a])*(eigval[k]-eigval[c]-eigval[i]+eigval[a]))

$( \epsilon_{j} - \epsilon_{b} - \epsilon_{i} + \epsilon_{a})(\epsilon_{k} - \epsilon_{c} + \epsilon_{i} +\epsilon_{a})$

In [4]:
def denominator_2(eigval,i,j,k,a,b,c):
    return ((eigval[j]-eigval[b]-eigval[i]+eigval[a])*(eigval[k]-eigval[c]+eigval[i]+eigval[a]))

$( \epsilon_{j} - \epsilon_{b} + \epsilon_{i} + \epsilon_{a})(\epsilon_{k} - \epsilon_{c} - \epsilon_{i} +\epsilon_{a})$

In [5]:
def denominator_3(eigval,i,j,k,a,b,c):
    return ((eigval[j]-eigval[b]+eigval[i]+eigval[a])*(eigval[k]-eigval[c]-eigval[i]+eigval[a]))

$( \epsilon_{j} - \epsilon_{b} + \epsilon_{i} + \epsilon_{a})(\epsilon_{k} - \epsilon_{c} + \epsilon_{i} +\epsilon_{a})$

In [6]:
def denominator_4(eigval,i,j,k,a,b,c):
    return ((eigval[j]-eigval[b]+eigval[i]-eigval[a])*(eigval[k]-eigval[c]+eigval[i]-eigval[a]))

Term 1:
$$\frac{ \sum_{ia,jb,kc} \langle ij|ab\rangle\langle jk|bc\rangle\langle ki|ca\rangle }{( \epsilon_{j} - \epsilon_{b} - \epsilon_{i} + \epsilon_{a})(\epsilon_{k} - \epsilon_{c} - \epsilon_{i} +\epsilon_{a})} $$

where $i,j,k$ are occupied and $a,b,c$ are unoccupied orbitals.

In [7]:
def Term_1(eri,eigval,M,N):
    T = 0 
    for i in range(0,N):
        for j in range(0,N):
            for k in range(0,N):
                for a in range(N,M):
                    for b in range(N,M):
                        for c in range(N,M):
                            print(i,j,k,a,b,c , enumerator(eri,i,j,k,a,b,c),\
                                  denominator_1(eigval,i,j,k,a,b,c), denominator_2(eigval,i,j,k,a,b,c),\
                                  denominator_3(eigval,i,j,k,a,b,c), denominator_4(eigval,i,j,k,a,b,c))

    return T

In [8]:
mol = gto.Mole()
mol.atom = """
    He    0.    0.    0.
"""
mol.basis = "cc-pvdz"
#mol.basis = "6-31g" 
mol.build()

In [9]:
# Run Hartree-Fock.
mf = scf.RHF(mol)
mf.kernel()
h = mf.get_hcore()
C = mf.mo_coeff
h1 = C.T@h@C
epsilon = mf.mo_energy

converged SCF energy = -2.85516047724274


In [10]:
# Find electron-repulsion integrals (eri).
eri = ao2mo.kernel(mol, mf.mo_coeff)
eri = np.asarray(ao2mo.restore(1, eri, mol.nao))

In [11]:
print(eri.shape)

(5, 5, 5, 5)


In [12]:
N = mol.nelec[0]
M = mol.nao
print(M,N)
print(Term_1(eri,epsilon,M,N))

5 1
0 0 0 1 1 1 0.011775347836616646 0.0 -0.0 -0.0 21.373786473242507
0 0 0 1 1 2 0.0 -0.0 -0.0 2.0603620324456053 26.583787364148705
0 0 0 1 1 3 0.0 -0.0 -0.0 2.0603620324456053 26.583787364148705
0 0 0 1 1 4 0.0 -0.0 -0.0 2.0603620324456053 26.583787364148705
0 0 0 1 2 1 0.0 -0.0 2.0603620324456053 -0.0 26.583787364148705
0 0 0 1 2 2 0.0 1.2699719498566482 3.3303339823022533 3.3303339823022533 33.06376020491155
0 0 0 1 2 3 0.0 1.2699719498566482 3.3303339823022533 3.3303339823022533 33.06376020491155
0 0 0 1 2 4 0.0 1.2699719498566482 3.3303339823022533 3.3303339823022533 33.06376020491155
0 0 0 1 3 1 0.0 -0.0 2.0603620324456053 -0.0 26.583787364148705
0 0 0 1 3 2 0.0 1.2699719498566482 3.3303339823022533 3.3303339823022533 33.06376020491155
0 0 0 1 3 3 0.0 1.2699719498566482 3.3303339823022533 3.3303339823022533 33.06376020491155
0 0 0 1 3 4 0.0 1.2699719498566482 3.3303339823022533 3.3303339823022533 33.06376020491155
0 0 0 1 4 1 0.0 -0.0 2.0603620324456053 -0.0 26.583787364148705
